# Concolic fuzzing

在信息流这一章中，我们已经看到了动态污染如何通过指示输入的哪一部分到达了有趣的地方来引导模糊。然而，动态污染跟踪所能传播的信息有限。例如，我们可能想了解当输入的某些属性发生变化时会发生什么?

In [ ]:
from fuzzingbook.fuzzingbook_utils.Coverage import Coverage
import inspect
import z3

In [ ]:
class ArcCoverage(Coverage):
    def traceit(self, frame, event, args):
        if event != 'return':
            f = inspect.getframeinfo(frame)
            self._trace.append((f.function, f.lineno))
        return self.traceit

    def arcs(self):
        t = [i for f, i in self._trace]
        return list(zip(t, t[1:]))

In [ ]:
def factorial(n):
    if n < 0:
        return None
    if n == 0:
        return 1
    if n == 1:
        return 1
    v = 1
    while n != 0:
        v = v * n
        n = n - 1
    return v

In [ ]:
with ArcCoverage() as cov:
    factorial(5)

In [119]:
cov.arcs()

[(1, 2),
 (2, 4),
 (4, 6),
 (6, 8),
 (8, 9),
 (9, 10),
 (10, 11),
 (11, 9),
 (9, 10),
 (10, 11),
 (11, 9),
 (9, 10),
 (10, 11),
 (11, 9),
 (9, 10),
 (10, 11),
 (11, 9),
 (9, 10),
 (10, 11),
 (11, 9),
 (9, 12),
 (12, 58),
 (58, 59)]

可以看到，有的路径没有执行。

一种方法是使用符号变量来表示输入，对约束进行编码，并使用SMT求解器来解决相应约束的否定。

## SMT 求解器

[SMT求解技术的发展及最新应用研究综述](http://crad.ict.ac.cn/CN/abstract/abstract3470.shtml)

官网： [z3 -- github](https://github.com/Z3Prover/z3) | [z3的document](https://github.com/Z3Prover/z3/wiki/Documentation)

非官网：[Z3Py Guide](https://ericpony.github.io/z3py-tutorial/guide-examples.htm) | [Programming Z3](https://theory.stanford.edu/~nikolaj/programmingz3.html)

`pip install z3-solver`

In [ ]:
x = z3.Int('x')
y = z3.Int('y')

s = z3.Solver()
print(s)

s.add(x > 10, y == x + 2)
print(s)
print("Solving constraints in the solver s ...")
if s.check() == z3.sat:
    print(s.model())
else:
    print("un-solved")

print("Create a new scope...")
s.push()
s.add(y < 11)
print(s)
print("Solving updated set of constraints...")
if s.check() == z3.sat:
    print(s.model())
else:
    print("un-solved")

print("Restoring state...")
s.pop()
print(s)
print("Solving restored set of constraints...")
if s.check() == z3.sat:
    print(s.model())
else:
    print("un-solved")

In [ ]:
# print(z3.get_version())
# set_option: Alias for 'set_param' for backward compatibility.
# set_param : Set Z3 global (or module) parameters
# 但是有哪些全局设置？找了一圈没有找见。
# 下面两个是，指定z3str3 solver，超时设置30 seconds
z3.set_option('smt.string_solver', 'z3str3')
z3.set_option('timeout', 30 * 1000)  # milliseconds

In [ ]:
# 对约束进行编码需要声明一个对应于输入n的符号变量。
zn = z3.Int('n')
predicates = [z3.Not(zn < 0), z3.Not(zn == 0)]
print(predicates)
z3.solve(predicates)

使用cov可以看出哪些路径没有执行.使用SMT解析器,给出相反的约束,执行没有执行的路径.如此迭代,使得每条路径得以覆盖.如果这个过程可以自动化实现,那会很棒.

## A Concolic Tracer

In [ ]:
class ConcolicTracer:
    def __init__(self,context=None):
        # context参数包含到目前为止看到的符号变量的声明，如果有前置条件，则包含前置条件。
        self.context = context if context is not None else ({},[])
        self.decls,self.path = self.context
    
    def __enter__(self):
        return self
    
    def __exit__(self,exc_type,exc_value,tb):
        return
    
    def __getitem__(self,fn):
        # 我们使用自省来确定函数的参数，该参数被连接到getitem方法中。
        # __getitem__,调用此方法以实现 self[key] 的求值
        # Python自省（反射）指南:https://www.cnblogs.com/huxi/archive/2011/01/02/1924317.html
        self.fn = fn
        self.fn_args = {i:None for i in inspect.signature(fn).parameters }
        return self
    
    def __call__(self,*args):
        # 此方法会在实例作为一个函数被“调用”时被调用；
        self.result = self.fn(*self.concolic(args))
        return self.result
    
    def concolic(self,args):
        # 稍后将修改它以生成符号变量。
        return args

In [ ]:
with ConcolicTracer() as _:
    # ConcolicTracer的对象为_;
    # 对象_调用__getitem__方法,使得self.fn=factorial
    # ()调用__call__方法
    # 参数1,经过concolic修改,传递给fn函数.
    # 执行结果保存在
    result = _[factorial](1)
print(result)
_.context

## Concolic Proxy Objects

[staticmethod的使用](https://www.pynote.net/archives/1388):介绍了staticmethod的使用,装饰器,闭包.很好的文章

[python的值传递和引用传递](https://www.cnblogs.com/CheeseZH/p/5165283.html):在Python中，数字、字符或者元组等不可变对象类型都属于值传递，而字典dict或者列表list等可变对象类型属于引用传递。

In [ ]:
def zproxy_create(cls,sname,z3var,context,zn,v=None):
    # 该方法给定一个类名，正确地创建该类的实例和相应的符号变量，并在上下文信息上下文中注册符号变量。
    zv = cls(context,z3var(zn),v) # 创建该类的实例;z3var(zn)为创建相应的符号变量;v是符号变量具体的值;
    context[0][zn] = sname # 在上下文中注册(引用传递);注意这里都是在第一个元素中修改的.
    return zv

### A Proxy Class for Booleans

In [ ]:
class zbool:
    # 下面这个类方法,在数据预处理这里(先预处理,载调用init方法),很漂亮!!!
    @classmethod
    def create(cls,context,zn,v):
        # (zbool类,)传入上下文环境,待转化为符号变量的名称(字符串),符号变量具体的值
        return zproxy_create(cls,'Bool',z3.Bool,context,zn,v)
    
    def __init__(self,context,z,v=None):
        # 注意这里并没有强制检查z为Boolen类型
        self.context, self.z, self.v = context,  z, v
        self.decl, self.path = self.context
    
    def __not__(self):
        # object中没有这个方法,https://docs.python.org/zh-cn/3/reference/datamodel.html
        # 所以不能not zbool对象
        # 这里仅仅是定义一个普通的方法
        return zbool(self.context,z3.Not(self.z),not self.v)
    
    def __bool__(self):
        # 调用此方法以实现真值检测以及内置的 bool() 操作
        # 这个对象可以用来判断Trur or False
        # 这里把谓词,注册到条件上
        r,pred = (True,self.z) if self.v else (False,z3.Not(self.z))
        self.path.append(pred)
        return r

In [118]:
with ConcolicTracer() as _:
    # za, zb = z3.Ints('a b')

    # 调用zbool的类方法create
    # z3.Bool:Return a Boolean constant named `name`. If `ctx=None`, then the global context is used.
    # 使用z3.Bool,创建名为my_bool_arg的Boolean类型常量,它具体的值为True
    # 将符号变量my_bool_arg,加入总环境中,它的类型是bool;(存储方式为字典)
    # 返回的对象中包含五个成员变量,见init方法
    val = zbool.create(_.context,'my_bool_arg',True)
    val_2 = val.__not__()

    if val:
        print("success")

print(val.z, val.v)
print(type(val.z), type(val.v))
print(_.context)
print(val.context)
assert id(_.context) == id(val.context)
print(val_2.z, val_2.v)

success
my_bool_arg True
<class 'z3.z3.BoolRef'> <class 'bool'>
({'my_bool_arg': 'Bool'}, [my_bool_arg])
({'my_bool_arg': 'Bool'}, [my_bool_arg])
Not(my_bool_arg) False


### A Proxy Class for Integers

In [ ]:
class zint(int):
    def __new__(cls,context,zn,v,*args,**kw):
        return int.__new__(cls,v,*args,**kw)
    
    @classmethod
    def create(cls,context,zn,v=None):
        # z3.Int:Return an integer constant named `name`. If `ctx=None`, then the global context is used.
        return zproxy_create(cls,'Int',z3.Int,context,zn,v)
    
    def __init__(self,context,z,v=None):
        self.z, self.v, self.context = z, v, context
    
    def __int__(self):
        return self.v
    
    def __pos__(self):
        # 调用此方法以实现一元算术运算+
        return self.v
    
    def _zv(self,o):
        return (o.z, o.v) if isinstance(o, zint) else (z3.IntVal(o), o) 
    
    def __ne__(self, other):
        z, v = self._zv(other)
        return zbool(self.context, self.z != z, self.v != v)

    def __eq__(self, other):
        z, v = self._zv(other)
        return zbool(self.context, self.z == z, self.v == v)
    
    def __req__(self, other):
        return self.__eq__(other)


    def __lt__(self, other):
        z, v = self._zv(other)
        return zbool(self.context, self.z < z, self.v < v)

    def __gt__(self, other):
        z, v = self._zv(other)
        return zbool(self.context, self.z > z, self.v > v)

    def __le__(self, other):
        z, v = self._zv(other)
        return zbool(self.context, z3.Or(self.z < z, self.z == z),
                     self.v < v or self.v == v)

    def __ge__(self, other):
        z, v = self._zv(other)
        return zbool(self.context, z3.Or(self.z > z, self.z == z),
                     self.v > v or self.v == v)
    
    def __bool__(self):
        # 对象存在则为真。并不判断self.v

        # 因为条件判断需要原始的布尔值。
        # 如果返回这个，就好了。这个再调用下自己的__bool__
        # return zbool(self.context, self.z, self.v) <-- not allowed

        # force registering boolean condition
        # 这调用上面的__ne__，返回一个zint对象
        # if zint对象，调用该对象的__bool__方法
        if self != 0:
            return True
        return False

In [ ]:
######## 整数的二元运算 ################
INT_BINARY_OPS = [
    '__add__',
    '__sub__',
    '__mul__',
    '__truediv__',
    # '__div__',
    '__mod__',
    # '__divmod__',
    '__pow__',
    # '__lshift__',
    # '__rshift__',
    # '__and__',
    # '__xor__',
    # '__or__',
    '__radd__',
    '__rsub__',
    '__rmul__',
    '__rtruediv__',
    # '__rdiv__',
    '__rmod__',
    # '__rdivmod__',
    '__rpow__',
    # '__rlshift__',
    # '__rrshift__',
    # '__rand__',
    # '__rxor__',
    # '__ror__',
]

def make_int_binary_wrapper(fname, fun, zfun):
    def proxy(self, other):
        z, v = self._zv(other)
        z_ = zfun(self.z, z)
        v_ = fun(self.v, v)
        if isinstance(v_, float):
            # we do not implement float results yet.
            assert round(v_) == v_
            v_ = round(v_)
        return zint(self.context, z_, v_)

    return proxy

def initialize():
    for fn in INITIALIZER_LIST:
        fn()


def init_concolic_1():
    for fname in INT_BINARY_OPS:
        fun = getattr(int, fname)
        # ArithRef Class Reference:https://z3prover.github.io/api/html/classz3py_1_1_arith_ref.html
        zfun = getattr(z3.ArithRef, fname)
        setattr(zint, fname, make_int_binary_wrapper(fname, fun, zfun))

###### 整数的一元运算 ###########
INT_UNARY_OPS = [
    '__neg__',
    '__pos__',
    # '__abs__',
    # '__invert__',
    # '__round__',
    # '__ceil__',
    # '__floor__',
    # '__trunc__',
]

def make_int_unary_wrapper(fname, fun, zfun):
    def proxy(self):
        return zint(self.context, zfun(self.z), fun(self.v))

    return proxy

def init_concolic_2():
    for fname in INT_UNARY_OPS:
        fun = getattr(int, fname)
        zfun = getattr(z3.ArithRef, fname)
        setattr(zint, fname, make_int_unary_wrapper(fname, fun, zfun))

# 目前需要初始化的之后init_concolic_1。后面多了，循环初始化。
INITIALIZER_LIST = []
INITIALIZER_LIST.append(init_concolic_1)
INITIALIZER_LIST.append(init_concolic_2)
init_concolic_1()
init_concolic_2()

In [ ]:
with ConcolicTracer() as _:
    val = zint.create(_.context, 'int_arg', 0)
    print(val.z, val.v)

    print(val._zv(0))
    print(val._zv(val))

    ia = zint.create(_.context, 'int_a', 0)
    ib = zint.create(_.context, 'int_b', 0)
    v1 = ia == ib
    v2 = ia != ib
    v3 = 0 != ib
    print(v1.z, v2.z, v3.z)


    ia = zint.create(_.context, 'int_a', 0)
    ib = zint.create(_.context, 'int_b', 1)
    v1 = ia > ib
    v2 = ia < ib
    print(v1.z, v2.z)
    v3 = ia >= ib
    v4 = ia <= ib
    print(v3.z, v4.z)


    ia = zint.create(_.context, 'int_a', 0)
    ib = zint.create(_.context, 'int_b', 1)
    print((ia + ib).z)
    print((ia + 10).z)
    print((11 + ib).z)
    print((ia - ib).z)
    print((ia * ib).z)
    print((ia / ib).z)
    print((ia ** ib).z)


    ia = zint.create(_.context, 'int_a', 0)
    print((-ia).z)
    print((+ia).z)

    za = zint.create(_.context, 'int_a', 1)
    zb = zint.create(_.context, 'int_b', 0)
    # 并没有给zint类，赋予__and__属性
    # 这里的运算逻辑是，za的__bool__,返回true；zb的__bool__，返回false
    # true and false
    if za and zb:
        print('1')

_.context

### Translating to the SMT Expression Format

[SMT-LIB官网地址](http://smtlib.cs.uiowa.edu/language.shtml) | [The SMT-LIBv2 Language and Tools: A Tutorial](http://smtlib.github.io/jSMTLIB/SMTLIBTutorial.pdf) | [The SMT-LIBv2 Language and Tools: A Tutorial 的中文翻译](https://tongtianta.site/paper/69887)

In [122]:
def triangle(a, b, c):
    if a == b:
        if b == c:
            return 'equilateral'
        else:
            return 'isosceles'
    else:
        if b == c:
            return 'isosceles'
        else:
            if a == c:
                return 'isosceles'
            else:
                return 'scalene'

In [123]:
class ConcolicTracer(ConcolicTracer):
    def smt_expr(self, show_decl=False, simplify=False, path=[]):
        r = []
        if show_decl:
            for decl in self.decls:
                v = self.decls[decl]
                v = '(_ BitVec 8)' if v == 'BitVec' else v
                r.append("(declare-const %s %s)" % (decl, v))
        path = path if path else self.path
        if path:
            path = z3.And(path)
            if show_decl:
                if simplify:
                    return '\n'.join([
                        *r,
                        "(assert %s)" % z3.simplify(path).sexpr()
                    ])
                else:
                    return '\n'.join(
                        [*r, "(assert %s)" % path.sexpr()])
            else:
                return z3.simplify(path).sexpr()
        else:
            return ''

In [127]:
with ConcolicTracer() as _:
    za = zint.create(_.context, 'int_a', 1)
    zb = zint.create(_.context, 'int_b', 1)
    zc = zint.create(_.context, 'int_c', 1)
    triangle(za, zb, zc)
print(_.context)
print(_.smt_expr(show_decl=True))
z3.solve(_.path)

({'int_a': 'Int', 'int_b': 'Int', 'int_c': 'Int'}, [int_a == int_b, int_b == int_c])
(declare-const int_a Int)
(declare-const int_b Int)
(declare-const int_c Int)
(assert (and (= int_a int_b) (= int_b int_c)))
[int_c = 0, int_a = 0, int_b = 0]
